In [1]:
cd ../..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras.layers import (Dense, Dropout, Flatten, Input, BatchNormalization, Lambda, 
                                     concatenate, Embedding, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import Callback, LearningRateScheduler

import wandb
from wandb.keras import WandbCallback

# own imports
from evaluation import Referee, load_data, select_dates, select_day_nums
from train import BatchCreator, Logger, plot_confidence_series, plot_some_confidence_intervals
from model_builder import (get_pinball_losses, get_simple_dist_model, get_simple_dense_model,
                           get_variable_dist_model, get_direct_dist_model)
from flow import model_predict, denorm_preds, warp_preds_to_ref_form, plot_some, evaluate_model
from preprocess import preprocess, get_features, pandas_cat_data, reset_categorical_features


# Setup file paths
data_dir = 'data/'
os.environ['DATA_DIR'] = data_dir
sub_dir = 'submissions_uncertainty/'
plt.rcParams['font.size'] = 13

# Hardcode requested quantiles
quantiles = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]

# Implement logic for obtaining the features for the validation set

In [3]:
# Load in the data set
calendar, sales_train_validation, sell_prices = load_data()

sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
sales_train = select_dates(sales_train_validation, day_start=1, num_days=1913-28, include_metadata=True)

ref = Referee(sales_true, sales_train, sell_prices, calendar, verbose=True)

Initializing Referee
Converting true sales to quantile form
Calculating weights for each level...
Calculating scale for each level...
Finished setup.


## Setup

For each level:  

1. Read the (prepared) dataset
2. Get the validation batch creator
3. Get the denormalised predictions
4. Save to file

Then:

1. Combine predictions

In [4]:
import os, gc
import numpy as np
import pandas as pd

from flow import restore_tags_converted_sales, read_converted_sales
from lightgbm_kernel import read_data, encode_categorical, reduce_mem_usage
from feature_extraction import aggregate_adapted_fe

from sklearn.preprocessing import LabelEncoder

In [5]:
# select whether to perform the category-reset step of the preprocessing
# reset_categories = {d: False if d in [3] else True for d in range(1, 1+12)}
reset_categories = {d: True for d in range(1, 1+12)}

In [6]:
from preprocess import categorical_features
from train import add_lgb_predictions
verbose = True
n_years = 6  # for correct categories
lgb_prediction = True
lgb_prediction_dir = 'models/uncertainty/final/run1/'
augment_events = False

In [18]:
for lag in [28, 21, 14, 7, 3, 2, 1]:
    for level in range(1, 1+9):
        # read data for pipeline from lightgbm-poisson-w-scaled-pinball-loss.ipynb
        calendar, sell_prices, sales_train_val, submission = read_data()  # with memory reduction

        # read comverted sales
        converted_sales = read_converted_sales(level=level, data_dir=data_dir)

        ### Replace demand with normalised sales
        sales_train_val = converted_sales

        ## Count
        NUM_ITEMS = sales_train_val.shape[0]  # 1 / ... / 70 / ... / 30,240

        nrows = int(365 * n_years * NUM_ITEMS)

        ## Encode categorical features
        calendar = encode_categorical(
            calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
        ).pipe(reduce_mem_usage)

        sales_train_val = encode_categorical(
            sales_train_val, categorical_features[level],
        ).pipe(reduce_mem_usage)

        sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]).pipe(
            reduce_mem_usage
        )

        ## Reshape
        sales_train_val = pd.melt(sales_train_val,
                                  id_vars=['id', *categorical_features[level]],
                                  var_name='day', value_name='demand')
        print('Melted sales train validation has {} rows and {} columns'.format(*sales_train_val.shape))

        #################################### ADD EVAL ###

        # select which days to add
        day_start = 1914
        day_end = 1941
        n_days = day_end - day_start + 1

        # select a 'prototype' for the new days
        df_bit = sales_train_val[sales_train_val.day == 'd_1913'].copy()
        df_bit['demand'] = -1
        df_bit = df_bit.reset_index(drop=True)

        # repeat the prototype for n days
        df_eval = pd.concat([df_bit]*n_days, ignore_index=True)

        # set the day to the corresponding values, e.g. d_1914, ..., d_1941
        validation_d_cols = ['d_%d'%d for d in range(day_start, day_end+1)]
        df_eval.day = np.repeat(validation_d_cols, df_bit.index.size)

        # merge with 'normal' train/val sales
        sales_train_val = pd.concat((sales_train_val, df_eval), ignore_index=True)
        #################################### END OF ADD EVAL ###

        print("Selecting {} rows ({:.1%})".format(nrows, nrows / sales_train_val.index.size))
        data = sales_train_val.iloc[-nrows:, :]

        ## Add calendar features
        # calendarの結合
        # drop some calendar features(不要な変数の削除:weekdayやwdayなどはdatetime変数から後ほど作成できる。)
        calendar.drop(['weekday', 'wday', 'month', 'year'],
                      inplace=True, axis=1)

        # notebook crash with the entire dataset (maybe use tensorflow, dask, pyspark xD)(dayとdをキーにdataに結合)
        data = pd.merge(data, calendar, how='left', left_on=['day'], right_on=['d'])
        data.drop(['d', 'day'], inplace=True, axis=1)

        # add sell price if all of the columns 'store_id', 'item_id', 'wm_yr_wk' are available
        # sell price
        if np.prod([col in data.columns for col in ['store_id', 'item_id', 'wm_yr_wk']]):
            # get the sell price data (this feature should be very important)
            data = data.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
            print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

        # memory
        del sell_prices
        gc.collect()

        data = reduce_mem_usage(aggregate_adapted_fe(data, DAYS_PRED=lag))

        if reset_categories[level]:
            print("Converting to pandas data..") if verbose else None
            # set categorical features
            data, available_cat_features = pandas_cat_data(data)

            print("Resetting categories..") if verbose else None
            # reset categorical features, set NaN events as additional category, set NaN shift/std/mean/kurt/skew to zero
            data = reset_categorical_features(data, available_cat_features)

        # select features
        features = get_features(level=level, prediction_lag=lag,
                                sell_price_features=('sell_price' in data.columns))


        if lgb_prediction:
            print("Adding lgb prediction as feature") if verbose else None
            data, features = add_lgb_predictions(data, level, features, lgb_prediction_dir, prediction_lag=lag)


        print(features) if verbose else None

        #################################### SAVE EVAL ###
        eval_df = data[(data['date'] >= '2016-04-24') & (data['date'] <= '2016-05-22')]

        aug_fn_part = "_aug_events" if augment_events else ""
        fn = data_dir + 'eval_prep/level_{}_simple_fe{}_{}_{}_normalised_demand_lag_{}.pickle'.format(
            level, aug_fn_part, eval_df.date.min().date().strftime("%Y_%m_%d"), eval_df.date.max().date().strftime("%Y_%m_%d"),
            lag
        )
        print("Saving to file..")
        eval_df.to_pickle(fn)
        print("Finished.")

        del data
        gc.collect()

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.00 Mb (74.3% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 1913 rows and 3 columns
Selecting 2190 rows (112.8%)
Mem. usage decreased to  0.15 Mb (53.6% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level1_lag28.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t28', 'shift_t29', 'shift_t30', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.08 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.08 Mb (0.5% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 40173 rows and 6 columns
Selecting 45990 rows (112.8%)
Mem. usage decreased to  3.34 Mb (52.7% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level7_lag28.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t28', 'shift_t29', 'shift_t30', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180'

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.01 Mb (74.7% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.01 Mb (0.2% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 5739 rows and 4 columns
Selecting 6570 rows (112.8%)
Mem. usage decreased to  0.47 Mb (53.3% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level4_lag21.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t21', 'shift_t22', 'shift_t23', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.00 Mb (74.3% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 1913 rows and 3 columns
Selecting 2190 rows (112.8%)
Mem. usage decreased to  0.15 Mb (53.6% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level1_lag14.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t14', 'shift_t15', 'shift_t16', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 

Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.08 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.08 Mb (0.5% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 40173 rows and 6 columns
Selecting 45990 rows (112.8%)
Mem. usage decreased to  3.34 Mb (52.7% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level7_lag14.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t14', 'shift_t15', 'shift_t16', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year

...

Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to  0.00 Mb (74.3% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 1913 rows and 3 columns
Selecting 2190 rows (112.8%)
Mem. usage decreased to  0.15 Mb (53.6% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level1_lag3.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t3', 'shift_t4', 'shift_t5', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end',

Mem. usage decreased to  0.08 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.08 Mb (0.5% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 40173 rows and 6 columns
Selecting 45990 rows (112.8%)
Mem. usage decreased to  3.34 Mb (52.7% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level7_lag3.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t3', 'shift_t4', 'shift_t5', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start

Mem. usage decreased to  0.01 Mb (74.7% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.01 Mb (0.2% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 5739 rows and 4 columns
Selecting 6570 rows (112.8%)
Mem. usage decreased to  0.47 Mb (53.3% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level4_lag2.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t2', 'shift_t3', 'shift_t4', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start',

Mem. usage decreased to  0.00 Mb (74.3% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 1913 rows and 3 columns
Selecting 2190 rows (112.8%)
Mem. usage decreased to  0.15 Mb (53.6% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level1_lag1.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t1', 'shift_t2', 'shift_t3', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start',

Mem. usage decreased to  0.08 Mb (0.5% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 40173 rows and 6 columns
Selecting 45990 rows (112.8%)
Mem. usage decreased to  3.34 Mb (52.7% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level7_lag1.csv..
['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t1', 'shift_t2', 'shift_t3', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start', 'is_month_end', 'is_month_start', 'is_weekend', 'state_id', 'cat_id', 'dept_id', 'lgb_pred']
Saving

In [19]:
for lag in [28, 21, 14, 7, 3, 2, 1]:
    for level in range(10, 12+1):

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to 11.22 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to 11.16 Mb (0.5% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 5832737 rows and 6 columns
Selecting 6677310 rows (112.8%)
Mem. usage decreased to 491.02 Mb (52.5% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level10_lag28.csv..


/Users/joeranbosma/opt/anaconda3/envs/ismi/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t28', 'shift_t29', 'shift_t30', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start', 'is_month_end', 'is_month_start', 'is_weekend', 'cat_id', 'dept_id', 'item_id', 'lgb_pred']
Saving to file..
Finished.
Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to 33.72 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to 33.49 Mb (0.

KeyboardInterrupt: 

In [7]:
lag = 28
level = 12
n_years = 2
# read data for pipeline from lightgbm-poisson-w-scaled-pinball-loss.ipynb
calendar, sell_prices, sales_train_val, submission = read_data()  # with memory reduction

# read comverted sales
converted_sales = read_converted_sales(level=level, data_dir=data_dir)

### Replace demand with normalised sales
sales_train_val = converted_sales

## Count
NUM_ITEMS = sales_train_val.shape[0]  # 1 / ... / 70 / ... / 30,240

nrows = int(365 * n_years * NUM_ITEMS)

## Encode categorical features
calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales_train_val = encode_categorical(
    sales_train_val, categorical_features[level],
).pipe(reduce_mem_usage)

sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)

## Reshape
sales_train_val = pd.melt(sales_train_val,
                          id_vars=['id', *categorical_features[level]],
                          var_name='day', value_name='demand')
print('Melted sales train validation has {} rows and {} columns'.format(*sales_train_val.shape))

#################################### ADD EVAL ###

# select which days to add
day_start = 1914
day_end = 1941
n_days = day_end - day_start + 1

# select a 'prototype' for the new days
df_bit = sales_train_val[sales_train_val.day == 'd_1913'].copy()
df_bit['demand'] = -1
df_bit = df_bit.reset_index(drop=True)

# repeat the prototype for n days
df_eval = pd.concat([df_bit]*n_days, ignore_index=True)

# set the day to the corresponding values, e.g. d_1914, ..., d_1941
validation_d_cols = ['d_%d'%d for d in range(day_start, day_end+1)]
df_eval.day = np.repeat(validation_d_cols, df_bit.index.size)

# merge with 'normal' train/val sales
sales_train_val = pd.concat((sales_train_val, df_eval), ignore_index=True)
#################################### END OF ADD EVAL ###

print("Selecting {} rows ({:.1%})".format(nrows, nrows / sales_train_val.index.size))
data = sales_train_val.iloc[-nrows:, :]

## Add calendar features
# calendarの結合
# drop some calendar features(不要な変数の削除:weekdayやwdayなどはdatetime変数から後ほど作成できる。)
calendar.drop(['weekday', 'wday', 'month', 'year'],
              inplace=True, axis=1)

# notebook crash with the entire dataset (maybe use tensorflow, dask, pyspark xD)(dayとdをキーにdataに結合)
data = pd.merge(data, calendar, how='left', left_on=['day'], right_on=['d'])
data.drop(['d', 'day'], inplace=True, axis=1)

# add sell price if all of the columns 'store_id', 'item_id', 'wm_yr_wk' are available
# sell price
if np.prod([col in data.columns for col in ['store_id', 'item_id', 'wm_yr_wk']]):
    # get the sell price data (this feature should be very important)
    data = data.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

# memory
del sell_prices
gc.collect()

data = reduce_mem_usage(aggregate_adapted_fe(data, DAYS_PRED=lag))

if reset_categories[level]:
    print("Converting to pandas data..") if verbose else None
    # set categorical features
    data, available_cat_features = pandas_cat_data(data)

    print("Resetting categories..") if verbose else None
    # reset categorical features, set NaN events as additional category, set NaN shift/std/mean/kurt/skew to zero
    data = reset_categorical_features(data, available_cat_features)

# select features
features = get_features(level=level, prediction_lag=lag,
                        sell_price_features=('sell_price' in data.columns))


if lgb_prediction:
    print("Adding lgb prediction as feature") if verbose else None
    data, features = add_lgb_predictions(data, level, features, lgb_prediction_dir, prediction_lag=lag)


print(features) if verbose else None

#################################### SAVE EVAL ###
eval_df = data[(data['date'] >= '2016-04-24') & (data['date'] <= '2016-05-22')]

aug_fn_part = "_aug_events" if augment_events else ""
fn = data_dir + 'eval_prep/level_{}_simple_fe{}_{}_{}_normalised_demand_lag_{}.pickle'.format(
    level, aug_fn_part, eval_df.date.min().date().strftime("%Y_%m_%d"), eval_df.date.max().date().strftime("%Y_%m_%d"),
    lag
)
print("Saving to file..")
eval_df.to_pickle(fn)
print("Finished.")

del data
gc.collect()

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Mem. usage decreased to 112.65 Mb (74.8% reduction)
Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to 111.66 Mb (0.9% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)
Melted sales train validation has 58327370 rows and 8 columns
Selecting 22257700 rows (37.6%)
Our final dataset to train has 22257700 rows and 17 columns
Mem. usage decreased to 2186.34 Mb (48.2% reduction)
Converting to pandas data..
Resetting categories..
Adding lgb prediction as feature
Adding LightGBM prediction from models/uncertainty/final/run1/predictions_level12_lag28.csv..


/Users/joeranbosma/opt/anaconda3/envs/ismi/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'shift_t28', 'shift_t29', 'shift_t30', 'rolling_std_t7', 'rolling_std_t30', 'rolling_std_t60', 'rolling_std_t90', 'rolling_std_t180', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90', 'rolling_mean_t180', 'rolling_skew_t30', 'rolling_kurt_t30', 'year', 'month', 'week', 'day', 'dayofweek', 'is_year_end', 'is_year_start', 'is_quarter_end', 'is_quarter_start', 'is_month_end', 'is_month_start', 'is_weekend', 'state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'sell_price', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'lgb_pred']
Saving to file..
Finished.


0